# Data Prep

The goal of this notebook is to prep the data for the alignment task.  This includes computing audio features and generating a query list.

In [1]:
import numpy as np
import librosa as lb
import os
import os.path
from pathlib import Path
import multiprocessing

In [2]:
ANNOTATIONS_ROOT = Path('Chopin_Mazurkas/annotations_beat')
AUDIO_ROOT = Path('Chopin_Mazurkas/wav_22050_mono')
FEATURES_ROOT = Path('features')
train_files = Path('cfg_files/filelist.train.txt')
test_files = Path('cfg_files/filelist.test.txt')

In [3]:
if not os.path.exists(FEATURES_ROOT):
    os.mkdir(FEATURES_ROOT)

### Compute features on clean audio

First we compute features on the audio.

In [4]:
def compute_chroma_single(infile, outfile, sr = 22050, hop_length=512):
    y, sr = lb.core.load(infile, sr = sr)
    #F = lb.feature.chroma_cens(y, sr=sr, hop_length=hop_length)
    F = lb.feature.chroma_cqt(y, sr=sr, hop_length=hop_length, norm=2)
    np.save(outfile, F)
    return

In [5]:
def compute_chroma_batch(filelist, outdir, n_cores):
    
    # prep inputs for parallelization
    inputs = []
    with open(filelist, 'r') as f:
        for line in f:
            relpath = line.strip()
            reldir, fileid = os.path.split(relpath)
            featdir = outdir / reldir
            featdir.mkdir(parents=True, exist_ok=True)
            featfile = (featdir / fileid).with_suffix('.npy')
            audiofile = (AUDIO_ROOT / relpath).with_suffix('.wav')
            if os.path.exists(featfile):
                print(f"Skipping {featfile}")
            else:
                inputs.append((audiofile, featfile))

    # process files in parallel
    pool = multiprocessing.Pool(processes = n_cores)
    pool.starmap(compute_chroma_single, inputs)
    
    return

In [6]:
FEATS_CLEAN_DIR = FEATURES_ROOT / 'clean'
compute_chroma_batch(train_files, FEATS_CLEAN_DIR, 24)
compute_chroma_batch(test_files, FEATS_CLEAN_DIR, 24)

Skipping features/clean/Chopin_Op017No4/Chopin_Op017No4_Afanassiev-2001_pid9130-01.npy
Skipping features/clean/Chopin_Op017No4/Chopin_Op017No4_Ashkenazy-1981_pid9058-13.npy
Skipping features/clean/Chopin_Op017No4/Chopin_Op017No4_Beliavsky-2004_pid9152-13.npy
Skipping features/clean/Chopin_Op017No4/Chopin_Op017No4_Ben-Or-1989_pid9161-12.npy
Skipping features/clean/Chopin_Op017No4/Chopin_Op017No4_Biret-1990_pid9062-13.npy
Skipping features/clean/Chopin_Op017No4/Chopin_Op017No4_Blet-2003_pid9103-07.npy
Skipping features/clean/Chopin_Op017No4/Chopin_Op017No4_Block-1995_pid9064-04.npy
Skipping features/clean/Chopin_Op017No4/Chopin_Op017No4_Brailowsky-1960_pid9060-13.npy
Skipping features/clean/Chopin_Op017No4/Chopin_Op017No4_Chiu-1999_pid9048-13.npy
Skipping features/clean/Chopin_Op017No4/Chopin_Op017No4_Clidat-1994_pid9067-13.npy
Skipping features/clean/Chopin_Op017No4/Chopin_Op017No4_Cohen-1997_pid9105-13.npy
Skipping features/clean/Chopin_Op017No4/Chopin_Op017No4_Coop-1987_pid9159-04.npy

Skipping features/clean/Chopin_Op024No2/Chopin_Op024No2_Ashkenazy-1981_pid9058-15.npy
Skipping features/clean/Chopin_Op024No2/Chopin_Op024No2_Ax-1985_pid9179-07.npy
Skipping features/clean/Chopin_Op024No2/Chopin_Op024No2_Bacha-1998_pid9166e-08.npy
Skipping features/clean/Chopin_Op024No2/Chopin_Op024No2_Barbosa-1983_pid9174-15.npy
Skipping features/clean/Chopin_Op024No2/Chopin_Op024No2_Ben-Or-1989_pid9161-13.npy
Skipping features/clean/Chopin_Op024No2/Chopin_Op024No2_Biret-1990_pid9062-15.npy
Skipping features/clean/Chopin_Op024No2/Chopin_Op024No2_Brailowsky-1960_pid9060-15.npy
Skipping features/clean/Chopin_Op024No2/Chopin_Op024No2_Chiu-1999_pid9048-15.npy
Skipping features/clean/Chopin_Op024No2/Chopin_Op024No2_Clidat-1994_pid9067-15.npy
Skipping features/clean/Chopin_Op024No2/Chopin_Op024No2_Cohen-1997_pid9105-15.npy
Skipping features/clean/Chopin_Op024No2/Chopin_Op024No2_Cortot-1951_pid9066-15.npy
Skipping features/clean/Chopin_Op024No2/Chopin_Op024No2_Csalog-1996_pid1263-15.npy
Skip

### Generate query list

Here we generate a file containing each pair of files to be aligned.

In [7]:
def generate_query_list(filelist, outfile):
    
    # group files by piece
    d = {}
    with open(filelist, 'r') as f:
        for line in f:
            parts = line.strip().split('/')
            assert len(parts) == 2
            piece, fileid = parts
            if piece not in d:
                d[piece] = []
            d[piece].append(fileid)
            
    # print out all pairings
    with open(outfile, 'w') as fout:
        for piece in d:
            num_recordings = len(d[piece])
            for i in range(num_recordings):
                fileid1 = d[piece][i]
                for j in range(i+1, num_recordings):
                    fileid2 = d[piece][j]
                    line = f'{piece}/{fileid1} {piece}/{fileid2}\n'
                    fout.write(line)
                    
    return

In [8]:
train_queries = 'cfg_files/query.train.list'
test_queries = 'cfg_files/query.test.list'
generate_query_list(train_files, train_queries)
generate_query_list(test_files, test_queries)